In [23]:
import pandas as pd
import numpy as np

In [24]:
# Define some constants
latitude = [40.79736, 41.78701, 30.1444, 25.7738]
longitude = [-73.97785, -87.77166, -97.66876, -80.1936]
cities = ["ny", "il", "tx", "fl"]
start_date = "2016-01-01"
end_date = "2024-03-24"

In [28]:
# Unpickle the DataFrames
city_history_dfs = []

for i in range(len(cities)):
  city_history_dfs.append(pd.read_pickle("./merged_df_" + cities[i] + ".pkl"))

print("Loaded DFs for " + str(len(city_history_dfs)) + " cities.\n")
# print(city_history_dfs[0].info())
print(city_history_dfs[0].tail())

Loaded DFs for 4 cities.

            date  tmax_vc  tmin_vc  humi_vc  wind_vc  tmax_om  tmin_om  \
1454  2024-03-17     61.8     51.0     43.6     19.6   15.493    7.393   
1455  2024-03-18     49.8     39.8     36.9     15.1    9.943    4.093   
1456  2024-03-19     47.5     35.8     42.6     14.5    7.343    0.593   
1457  2024-03-20     55.7     37.1     51.3     17.1   11.143    1.443   
1458  2024-03-21     41.9     30.0     31.6     18.3    6.043   -3.457   

        sund_om  prec_om    wind_om  ...  tmin_ncei  wt08_ncei  wt01_ncei  \
1454  36546.020      2.0  28.440000  ...         48        NaN        NaN   
1455  34802.510      0.0  19.995398  ...         38        NaN        NaN   
1456  30800.836      0.0  26.980793  ...         36        NaN        NaN   
1457  26404.998      6.0  28.019936  ...         34        NaN        NaN   
1458  40807.540      0.0  28.793250  ...         30        NaN        NaN   

      wt02_ncei  wt06_ncei  wt04_ncei  tsun_ncei  pgtm_ncei  wt03_

In [29]:
for i in range(len(cities)):
  # Figure out what all cols to keep
  allCols = city_history_dfs[i].columns.tolist()
  print(len(allCols))
  print(*allCols, sep = '\n')
  # Keep the columns we need
  # date, tmax from all sources, 
  # humidity from visual crossing,
  # precipitation from open meteo,
  # snow from meteo stats
  # sunny time from ncei
  ny_data = city_history_dfs[i][['date', 'tmax_vc', 'tmax_om', 'tmax_ms', 'tmax_ncei', 'humi_vc', 'prec_om', 'tmin_ms', 'tmin_ncei']]
  # ny_data = city_history_dfs[i][['date', 'tmax_vc', 'tmax_om', 'tmax_ms', 'humi_vc', 'prec_om', 'tmin_ms', ]]
  ny_data.info()
  
  def cleanAndPreprocessData(df):
    columns_to_convert_to_farhenheit = ['tmax_om', 'tmax_ms', 'tmin_ms']
    for column in columns_to_convert_to_farhenheit:
      df[column] = df[column] * 9/5 + 32
    print(df.head())
    df['date'] = pd.to_datetime(df['date']) 
    df['day'] = df['date'].dt.dayofyear
    df['tmax_avg'] = df[['tmax_vc', 'tmax_om', 'tmax_ms' ]].mean(axis=1)
    df['tmin_avg'] = df[['tmin_ms']].mean(axis=1)
    return df

  ny_data = cleanAndPreprocessData(ny_data)
  ny_data.to_pickle("./data_cleaned_"+ cities[i] +".pkl")  

48
date
tmax_vc
tmin_vc
humi_vc
wind_vc
tmax_om
tmin_om
sund_om
prec_om
wind_om
tavg_ms
tmin_ms
tmax_ms
prcp_ms
snow_ms
wdir_ms
wspd_ms
wpgt_ms
pres_ms
tsun_ms
unnamed: 0_ncei
wsf2_ncei
station_ncei
wsf5_ncei
snow_ncei
rhmx_ncei
aslp_ncei
prcp_ncei
rhav_ncei
snwd_ncei
astp_ncei
rhmn_ncei
wdf2_ncei
awnd_ncei
wdf5_ncei
awbt_ncei
tmax_ncei
adpt_ncei
tmin_ncei
wt08_ncei
wt01_ncei
wt02_ncei
wt06_ncei
wt04_ncei
tsun_ncei
pgtm_ncei
wt03_ncei
wt05_ncei
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       1459 non-null   object 
 1   tmax_vc    1459 non-null   float64
 2   tmax_om    1459 non-null   float64
 3   tmax_ms    1459 non-null   float64
 4   tmax_ncei  1459 non-null   int64  
 5   humi_vc    1459 non-null   float64
 6   prec_om    1459 non-null   float64
 7   tmin_ms    1459 non-null   float64
 8   tmin_ncei  1459 non-null   int64  
dtype

/var/folders/b9/fkf035lj1y786gv5xh300_qc0000gn/T/ipykernel_81338/3216503694.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column] * 9/5 + 32
/var/folders/b9/fkf035lj1y786gv5xh300_qc0000gn/T/ipykernel_81338/3216503694.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = pd.to_datetime(df['date'])
/var/folders/b9/fkf035lj1y786gv5xh300_qc0000gn/T/ipykernel_81338/3216503694.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 